 ## Facial Keypoint Detection
 
I have tried to make a very simple kernal, which would help anyone in getting started with this exercise.

Once you are comfortable with this, you can go on to experiment with other methods to improve the accuracy. 

**Have fun and do upvote if you find this kernel helpful.**

In [ ]:
#Import required libraries
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

#List the locations of input data and directories
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Load data to DataFrame

In [ ]:
#Unzip the train and test data

!unzip -o ../input/facial-keypoints-detection/test.zip
!unzip -o ../input/facial-keypoints-detection/training.zip

In [ ]:
#Read csv data into a DataFrame

train_data = pd.read_csv('./training.csv')
test_data = pd.read_csv('./test.csv')
id_lookup = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv')

In [ ]:
#Number of data samples in train and test set

len(train_data),len(test_data)

In [ ]:
#Show sample training data 
train_data.head().T

In [ ]:
#Show sample test data
test_data.head()

In [ ]:
#Show sample data from id_lookup file
id_lookup.head()

## Check for Null Values

In [ ]:
train_data.isnull().sum()

In [ ]:
#Drop null values
train_data_clean = train_data.dropna()

## Create Features and Labels for Training Model

In [ ]:
#Create list of feature and labels
images = []
labels = []

for i, sample in train_data_clean.iterrows():
    #Features
    img = np.array((sample['Image'].split(' ')), dtype=float)
    img = np.reshape(img, (96, 96, 1))
    images.append(img)
    #Labels
    labels.append(sample[0:30])    

## Plot sample image and markers

In [ ]:
# Define function to plot images and markers
def plot_image(img_id, img, lab, axis):
    axis.imshow(img[img_id], cmap='gray')
    axis.scatter(lab[img_id][0:30:2], lab[img_id][1:30:2], marker='x', c = 'red')

# Plot 5 sample images
fig = plt.figure(figsize=(18,6))
for i in range(5):
    ax = fig.add_subplot(1, 5, i+1, xticks=[], yticks=[])    
    plot_image(i, images, labels, ax)

## Creating a CNN model

In [ ]:
# Import all building blocks to build a CNN using Keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPool2D, Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU

In [ ]:
# Defining the architecture for CNN model

model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', input_shape=(96,96,1)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())

model.add(Conv2D(32, (3,3), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.1))
          
model.add(Conv2D(128, (3,3), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, (3,3), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())

model.add(Conv2D(256, (3,3), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.1))
          
model.add(Conv2D(512, (3,3), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())

model.add(Conv2D(512, (3,3), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

In [ ]:
#Compile model with optimizer and loss function

model.compile(optimizer='adam', 
             loss='mean_squared_error',
             metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
# Converting lists to array in order to be compatable with model.fit()
images = np.array(images, dtype=float)
labels = np.array(labels, dtype=float)

In [ ]:
#Fit the model to training data
history = model.fit(images, labels, epochs=500, validation_split=0.1)

## Plot Root Mean Squared Error (RMSE)

In [ ]:
history.history.keys()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
ax1.plot(history.history['root_mean_squared_error'])
ax1.plot(history.history['val_root_mean_squared_error'])
ax1.set_title('RMS Error')
ax1.set(xlabel='epoch', ylabel='rms')
ax1.legend(['train', 'val'])

ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_title('Loss')
ax1.set(xlabel='epoch', ylabel='rms')
ax2.legend(['train', 'val'])

## Predicting on Test set

In [ ]:
#Create test data
test_images = []

for i, sample in test_data.iterrows():
    #Features
    img = np.array((sample['Image'].split(' ')), dtype=float)
    img = np.reshape(img, (96, 96, 1))
    test_images.append(img) 
test_images = np.array(test_images, dtype=float)

In [ ]:
#Predict on test set
pred = model.predict(test_images)

## Visualising Test Predictions

In [ ]:
fig = plt.figure(figsize=(18,6))
for i in range(5):
    ax = fig.add_subplot(1, 5, i+1, xticks=[], yticks=[])
    plot_image(i, test_images, pred, ax)
plt.show()

## Generating Submission File

In [ ]:
row_ids = list(id_lookup['RowId'])
img_ids = list(id_lookup['ImageId']-1)
feature_names = list(id_lookup['FeatureName'])

feature_list = []
for feature in feature_names:
    feature_list.append(feature_names.index(feature))
    
predictions = []
for x,y in zip(img_ids, feature_list):
    predictions.append(pred[x][y])
    
row_ids = pd.Series(row_ids, name = 'RowId')
locations = pd.Series(predictions, name = 'Location').clip(0,96)

submission_result = pd.concat([row_ids,locations],axis = 1)
submission_result.to_csv('submission.csv',index = False)    